<a href="https://colab.research.google.com/github/vamshinr/colab-notebooks/blob/main/project3_215.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd

In [7]:
df_tweet = pd.read_csv("tweet_features.csv")

In [8]:
df_tweet.head(1)

,tweet_id,num_words,text_length,contains_questmark,num_questmark,contains_exclammark,num_exclammark,contains_happyemo,contains_sademo,contains_firstorderpron,contains_secondorderpron,contains_thirdorderpron,num_uppercasechars,num_possentiwords,num_negsentiwords,num_mentions,num_hashtags,num_URLs,num_retweets
0,263046056240115712,22,134,True,1,False,0,False,False,False,False,True,3,0.0,0.0,0,1,1,0


In [9]:
df_user = pd.read_csv("user_features.csv")

In [10]:
df_user.head(1)

,tweet_id,num_friends,num_followers,folfriend_ratio,times_listed,has_url,is_verified,num_tweets
0,263046056240115712,283,1651,5.833922,64,False,False,43811


In [11]:
len(df_user)

14483

In [12]:
data = []
with open("tweets.txt","r") as f:
    data = f.readlines()

In [13]:
tweets_data = []
for d in data[1:]:
    temp = d.strip("\n")
    temp = temp.split("\t")
    tweets_data.append(temp)

In [14]:
cols = data[0].strip("\n").split("\t")
df_tweets = pd.DataFrame(tweets_data,columns=cols)

In [15]:
df_tweets.head()

,tweetId,tweetText,userId,imageId(s),username,timestamp,label
0,263046056240115712,¿Se acuerdan de la película: “El día después d...,21226711,sandyA_fake_46,iAnnieM,Mon Oct 29 22:34:01 +0000 2012,fake
1,262995061304852481,@milenagimon: Miren a Sandy en NY! Tremenda i...,192378571,sandyA_fake_09,CarlosVerareal,Mon Oct 29 19:11:23 +0000 2012,fake
2,262979898002534400,"Buena la foto del Huracán Sandy, me recuerda a...",132303095,sandyA_fake_09,LucasPalape,Mon Oct 29 18:11:08 +0000 2012,fake
3,262996108400271360,Scary shit #hurricane #NY http://t.co/e4JLBUfH,241995902,sandyA_fake_29,Haaaaarryyy,Mon Oct 29 19:15:33 +0000 2012,fake
4,263018881839411200,My fave place in the world #nyc #hurricane #sa...,250315890,sandyA_fake_15,princess__natt,Mon Oct 29 20:46:02 +0000 2012,fake


In [16]:
df_tweets.label.value_counts()

fake     6841
real     5009
humor    2633
Name: label, dtype: int64

In [17]:
df_tweets = df_tweets[df_tweets['label']!='humor']

In [18]:
df_tweets.label.value_counts()

fake    6841
real    5009
Name: label, dtype: int64

In [19]:
df_tweets["imageId(s)"].value_counts()

sandyA_fake_29                   1051
sandyB_real_59                    634
sandyA_fake_11                    622
sandyB_real_71                    585
sandyA_fake_08                    413
                                 ... 
boston_fake_21                      1
boston_fake_11                      1
boston_fake_28                      1
boston_fake_33                      1
boston_fake_01,boston_fake_17       1
Name: imageId(s), Length: 353, dtype: int64

In [20]:
df_tweets.dropna(inplace=True)
df_tweets.reset_index(inplace=True)

In [21]:
# Pre-Processing
df_tweets['tweetText'] = df_tweets['tweetText'].apply(lambda x: x.lower())

In [22]:
df_tweets['target'] = df_tweets['label'].apply(lambda x: 0 if x == "fake" else 1)

In [23]:
df_tweets.target.value_counts()

0    6841
1    5009
Name: target, dtype: int64

In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [25]:
tokenizer = Tokenizer()

In [26]:
tokenizer.fit_on_texts(df_tweets['tweetText'].values)
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :- ",vocab_size)
X = tokenizer.texts_to_sequences(df_tweets['tweetText'].values)

Vocabulary Size :-  23535


In [27]:
max_length = 100
X = pad_sequences(X,maxlen = max_length, padding = 'post')

In [28]:

y = pd.get_dummies(df_tweets['label']).values

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=53)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(8295, 100) (8295, 2)
(3555, 100) (3555, 2)


In [31]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip

--2023-03-24 19:04:50--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-03-24 19:04:50--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-03-24 19:04:50--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [32]:
#!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [33]:
import numpy as np

embeddings_index = dict()
f = open('glove.6B.200d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))


embedding_matrix = np.zeros((vocab_size, 200))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

In [35]:
from keras.layers import Concatenate

In [29]:
# checkpoint= ModelCheckpoint(filepath="proj3_1.hdf5", verbose=1, save_best_only=True)

# monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=10, verbose=1, mode='auto')

# textmodel = Sequential()
# textmodel.add(Embedding(vocab_size, 200, weights=[embedding_matrix], input_length=max_length, trainable=False))
# textmodel.add(Flatten())
# textmodel.add(Dense(2, activation='sigmoid'))
# # compile the model
# textmodel.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
# # summarize the model
# print(textmodel.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 200)          4707000   
                                                                 
 flatten (Flatten)           (None, 20000)             0         
                                                                 
 dense (Dense)               (None, 2)                 40002     
                                                                 
Total params: 4,747,002
Trainable params: 40,002
Non-trainable params: 4,707,000
_________________________________________________________________
None


In [30]:
# model.fit(X_train, y_train, epochs=50, verbose=1, callbacks=[monitor,checkpoint], validation_data=(X_test,y_test))

Epoch 1/50
256/260 [============================>.] - ETA: 0s - loss: 0.2024 - accuracy: 0.6840

260/260 [==============================] - 2s 6ms/step - loss: 0.2021 - accuracy: 0.6849
Epoch 2/50
250/260 [===========================>..] - ETA: 0s - loss: 0.1594 - accuracy: 0.7839

260/260 [==============================] - 1s 6ms/step - loss: 0.1591 - accuracy: 0.7840
Epoch 3/50
254/260 [============================>.] - ETA: 0s - loss: 0.1423 - accuracy: 0.8081

260/260 [==============================] - 1s 5ms/step - loss: 0.1423 - accuracy: 0.8086
Epoch 4/50
257/260 [============================>.] - ETA: 0s - loss: 0.1315 - accuracy: 0.8281

260/260 [==============================] - 1s 5ms/step - loss: 0.1311 - accuracy: 0.8288
Epoch 5/50
253/260 [============================>.] - ETA: 0s - loss: 0.1238 - accuracy: 0.8391

...

260/260 [==============================] - 3s 11ms/step - loss: 0.0640 - accuracy: 0.9274
Epoch 47/50
257/260 [============================>.] - ETA: 0s - loss: 0.0639 - accuracy: 0.9267

260/260 [==============================] - 2s 8ms/step - loss: 0.0639 - accuracy: 0.9268
Epoch 48/50
254/260 [============================>.] - ETA: 0s - loss: 0.0636 - accuracy: 0.9285

260/260 [==============================] - 2s 7ms/step - loss: 0.0634 - accuracy: 0.9290
Epoch 49/50
253/260 [============================>.] - ETA: 0s - loss: 0.0628 - accuracy: 0.9298

260/260 [==============================] - 1s 5ms/step - loss: 0.0624 - accuracy: 0.9301
Epoch 50/50
255/260 [============================>.] - ETA: 0s - loss: 0.0627 - accuracy: 0.9297

260/260 [==============================] - 1s 5ms/step - loss: 0.0626 - accuracy: 0.9300


In [32]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 77.046412


In [ ]:
##concatenation

In [40]:
data = []
with open("images.txt","r") as f:
    data = f.readlines()

In [43]:
data[1].strip("\n").split("\t")

['boston_fake_01', 'http://i.imgur.com/LvoCC5T.jpg', 'fake', 'boston']

In [44]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [45]:
file_path = 'MediaEval2015_DevSet_Images.rar'
!unrar x $file_path


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from MediaEval2015_DevSet_Images.rar

Creating    Medieval2015_DevSet_Images                                OK
Creating    Medieval2015_DevSet_Images/BostonMarathon                 OK
Creating    Medieval2015_DevSet_Images/BostonMarathon/fakes           OK
Extracting  Medieval2015_DevSet_Images/BostonMarathon/fakes/boston_fake_01.jpg       0%  OK 
Extracting  Medieval2015_DevSet_Images/BostonMarathon/fakes/boston_fake_02.jpg       0%  OK 
Extracting  Medieval2015_DevSet_Images/BostonMarathon/fakes/boston_fake_03.jpg       0%  1%  OK 
Extracting  Medieval2015_DevSet_Images/BostonMarathon/fakes/boston_fake_04.jpg       1%  OK 
Extracting  Medieval2015_DevSet_Images/BostonMarathon/fakes/boston_fake_05.jpg       1%  2%  OK 
Extracting  Medieval2015_DevSet_Images/BostonMarathon/fakes/boston_fake_06.jpg       2%  OK 
Extracting  Medieval2015_Dev

In [46]:
import os

In [47]:
os.listdir("/content/Medieval2015_DevSet_Images")

['BostonMarathon',
 'Rock Elephant',
 'Underwater bedroom',
 'Sochi Olympics',
 'Pig Fish',
 'Malaysia Airlines',
 'Columbian Chemicals',
 'Passport',
 'Livr',
 'Bring Back Our Girls',
 'Hurricane Sandy']

In [48]:
# os.mkdir("dataset_images")
# os.mkdir("dataset_images/fake")
# os.mkdir("dataset_images/real")

In [49]:
import glob
import shutil
import os

In [50]:
from distutils.dir_util import copy_tree

In [51]:
# for path in os.listdir("/content/Medieval2015_DevSet_Images"):
#     dirs = os.listdir("/content/Medieval2015_DevSet_Images/"+path)
#     for d in dirs:
#         if d=="fakes":
#             copy_tree("/content/Medieval2015_DevSet_Images/"+path+"/"+d, "/content/dataset_images/fake")
#         elif d=="reals":
#             copy_tree("/content/Medieval2015_DevSet_Images/"+path+"/"+d, "/content/dataset_images/real")
#         else:
#           continue

In [52]:
batch_size = 32
img_height = 180
img_width = 180

data_dir = "dataset_images"

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 331 files belonging to 2 classes.
Using 265 files for training.


In [53]:
class_names = train_ds.class_names

In [54]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 180, 180, 3)
(32,)


In [55]:
normalization_layer = layers.Rescaling(1./255)

In [56]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


0.0006699506 1.0


In [62]:
num_classes = len(class_names)

cnnmodel = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [63]:
checkpoint= ModelCheckpoint(filepath="proj3_1.hdf5", verbose=1, save_best_only=True)

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=10, verbose=1, mode='auto')

textmodel = Sequential()
textmodel.add(Embedding(vocab_size, 200, weights=[embedding_matrix], input_length=max_length, trainable=False))
textmodel.add(Flatten())
textmodel.add(Dense(2, activation='sigmoid'))

In [68]:
#concatenate
model = Sequential()
model.add(Concatenate([cnnmodel, textmodel]))

In [ ]:
# compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
# summarize the model
print(model.summary())

In [ ]:
model.fit(X_train, y_train, epochs=50, verbose=1, callbacks=[monitor,checkpoint], validation_data=(X_test,y_test))